In [1]:
# -*- encoding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
from gensim import corpora,models,similarities,utils
import logging
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from keras.preprocessing import text,sequence

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
# 获取训练数据
def getTrainSet(inFile):
    # 文章标题集
    title_set = []
    # 训练集
    train_set=[]
    # 读入训练数据
    f=open(inFile)
    lines=f.readlines()
    for line in lines:
        article = line.replace('\n','').split('\t')
        title = article[0]
        title_set.append(title)
        content = article[1:]
        train_set.append(content)
    f.close()
    return (title_set,train_set)

In [5]:
# # 如果输入分布在硬盘上的多个文件中, 文件的每一行是一个句子, 那么可以逐个文件, 逐行的处理输入
# # 这样处理可以不必一次性把所有句子都载入内存
# class MySentences(object):
#     def __init__(self, dirname):
#         self.dirname = dirname
#     def __iter__(self):
#         for fname in os.listdir(self.dirname):
#             for line in open(os.path.join(self.dirname, fname)):
#                 yield line.split()

In [6]:
# # 引入新的数据在线训练,写入到新的模型里面
# def trainNew(newdata,modelFile,newModelFile):
#     new_model = models.Word2Vec.load(modelFile)
#     new_model.train(newdata)
#     # 存储模型
#     new_model.save(newModelFile)

In [7]:
# 计算单词相似度
def caluSimilarity(model,word):
    print u"*** %s ***" %(word)
    for x in model.most_similar(word,topn=10):
        print u"(%s,%s)" % (x[0].encode('utf-8'),x[1])

In [8]:
# 训练word2vec
def trainModel(inFile,modelFile,vecFile):
    # 读入数据    
    title_set,data_set = getTrainSet(inFile)
    
    # 训练
    # 少于min_count次数的单词会被丢弃掉, 默认值为5
    # size = 神经网络的隐藏层的单元数 default value is 100
    # workers= 控制训练的并行:default = 1 worker (no parallelization) 只有在安装了Cython后才有效
    model = models.Word2Vec(data_set,min_count=5,window=10,size = 50,workers=2)
    
    # 存储模型
    model.save(modelFile)
    
    # 存储vector
    model.wv.save_word2vec_format(vecFile, binary=True) 
    
    return model
    

In [9]:
# 测试
def testModel(modelFile,vecFile):
    # 装载模型
    model = models.Word2Vec.load(modelFile)
    
    word_vec = model.wv.load_word2vec_format(vecFile, binary=True) 
    
    print word_vec.vocab[u"金利"]
    # 获取相似词语
    caluSimilarity(model,"金利")
    
    caluSimilarity(model,"エネルギー")
    
    caluSimilarity(model,"外相")
    
    caluSimilarity(model,"家計")
    
    caluSimilarity(model,"株式")
    #获得词向量
#     print model["銀行"]

In [10]:
# 把原始文本转化为由词汇表索引表示的矩阵
def fastBuildSeq(inFile,outFile,modelFile,vecFile):
    # 读入数据
    title_set,data_set = getTrainSet(inFile)
    
    # 装载模型
    model = models.Word2Vec.load(modelFile)
    word_vec = model.wv.load_word2vec_format(vecFile, binary=True) 
    
    # 使用dir(object)查看对象的属性
    # 对每一个文章做转换      
    # 注意：由于word2vec的向量在训练的时候用的是unicode的编码，
    # 所以在字典里面匹配key的时候，需要把key转化为unicode的编码，使用decode('utf-8')
    transfrom = []
    for news in data_set:
        trs_news = [word_vec.vocab[w.decode('utf-8')].index for w in news if w.decode('utf-8') in word_vec.vocab]
#         # --- 调试
#         trs_news = []
#         for w in news:
#             if w.decode('utf-8') in word_vec.vocab:
#                 print "in vocab = ",w.decode('utf-8')
#                 trs_news.append((word_vec.vocab[w.decode('utf-8')].index,w))
#         # --
        transfrom.append(trs_news)
    
#     for x in transfrom:
#         print x
    
    # 对文字序列做补齐 ，补齐长度=最长的文章长度 ，补齐在最后，补齐用的词汇默认是词汇表index=0的词汇，也可通过value指定
    # 训练好的w2v词表的index = 0 对应的词汇是空格
    result_data = sequence.pad_sequences(transfrom,maxlen=200,padding='post')
    
#     print result_data
    
#     for x in result_data:
#         print x
    
    np.save(outFile,result_data)

In [13]:
def main():
    
#     inFile = "./data/test.txt"
    inFile = "./data/all.txt"
    outFile = "./data/train_cnn.npy"
#     modelFile = "./model/w2v.mdl"
    modelFile = "./model/w2v_dim50.mdl"
#     vecFile = "./model/vector.bin"
    vecFile = "./model/vector_dim50.bin"
    targetData = "./data/target.npy"
    
    # 读入标签
#     target = np.load(targetData)
    
    # 训练模型
#     trainModel(inFile,modelFile,vecFile)
    
    # 测试模型
    testModel(modelFile,vecFile)
    
    # 把分词以后的文本转化为供CNN训练的数据文件
#     fastBuildSeq(inFile,outFile,modelFile,vecFile)
    

In [14]:
if __name__ == '__main__':
    main()